In [25]:
import pickle
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression

In [2]:
PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
DATASET_FILE = "all_features"

In [3]:
# Load dataset
dataset = pickle.load(open(f"{PROCESSED_DATA_FOLDER}/{DATASET_FILE}.pkl", "rb"))

In [35]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192867 entries, 52886 to 195083
Columns: 7323 entries, city_resolvedAddress to target
dtypes: Sparse[float64, 0](7281), bool(1), datetime64[ns](1), float64(38), object(2)
memory usage: 1.3+ GB


In [33]:
# check the size of the data
dataset.shape

(192867, 7323)

In [34]:
# check few lines
dataset.head()

,city_resolvedAddress,day_datetime,isw_date_tomorrow_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,isw_zolota,isw_zolotarivka,isw_zone,isw_zorya,isw_ztpp,isw_zvanivka,isw_zvezda,isw_zybinski,isw_zyuganov,target
52886,"Чернігів, Україна",2022-02-25,2022-02-25,6.4,-1.7,2.1,-2.1,75.5,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
121198,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
121199,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
121200,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
121201,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [32]:
# check nan
dataset.isna().sum()

city_resolvedAddress          0
day_datetime                  0
isw_date_tomorrow_datetime    0
day_tempmax                   0
day_tempmin                   0
                             ..
isw_zvanivka                  0
isw_zvezda                    0
isw_zybinski                  0
isw_zyuganov                  0
target                        0
Length: 7323, dtype: int64

In [7]:
# Sort dataset by date
dataset['day_datetime'] = pd.to_datetime(dataset['day_datetime'])
dataset = dataset.sort_values(by='day_datetime')
dataset.day_datetime.describe(datetime_is_numeric=True)

count                           195084
mean     2022-08-03 16:45:45.451190272
min                2022-02-24 00:00:00
25%                2022-05-10 00:00:00
50%                2022-08-03 00:00:00
75%                2022-10-26 00:00:00
max                2023-01-20 00:00:00
Name: day_datetime, dtype: object

In [9]:
# Remove this line
dataset = dataset.dropna()

In [11]:
dataset['target'] = dataset['event_indicator']
dataset = dataset.drop(columns=['event_indicator'])

# check the event proportion of the data
dataset['target'].value_counts()

<ipython-input-11-22a786c5625e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['target'] = dataset['event_indicator']


False    143773
True      49094
Name: target, dtype: int64

In [67]:
# set an excluded list
exclude_list = ['city_resolvedAddress', 'target']

In [18]:
min_date = dataset['day_datetime'].min()
max_date = dataset['day_datetime'].max()
print("Min:", min_date, "Max:", max_date)

Min: 2022-02-25 00:00:00 Max: 2023-01-20 00:00:00


In [21]:
train_percent = .75
time_between = max_date - min_date
train_cutoff = min_date + train_percent*time_between
train_cutoff

Timestamp('2022-10-29 18:00:00')

In [22]:
train_df = dataset[dataset.day_datetime <= train_cutoff]
test_df = dataset[dataset.day_datetime > train_cutoff]

print("Train:", train_df.day_datetime.min(), train_df.day_datetime.max())
print("Test:", test_df.day_datetime.min(), test_df.day_datetime.max())

Train: 2022-02-25 00:00:00 2022-10-29 00:00:00
Test: 2022-10-30 00:00:00 2023-01-20 00:00:00


In [23]:
days_between = time_between / np.timedelta64(1, 'D')
days_between

329.0

In [24]:
# Train logistic regression model
model = LogisticRegression()

NameError: name 'LogisticRegression' is not defined

In [26]:
# Split the data into train and test sets using a rolling window
window_size = 100 # set the size of each test set
n_splits = len(df) // window_size # calculate the number of splits we'll need
tscv = TimeSeriesSplit(n_splits=n_splits)

for train_index, test_index in tscv.split(dataset):
    X_train, X_test = dataset.iloc[train_index], dataset.iloc[test_index]
    y_train, y_test = X_train.pop('target'), X_test.pop('target')

    # Train model using X_train and y_train
    model.fit(X_train, y_train)

    # Evaluate model on the test set
    accuracy = model.score(X_test, y_test)
    print(f"Accuracy: {accuracy}")

TypeError: object of type 'float' has no len()

In [28]:
# Split the data into train and test sets using a rolling window
window_size = 100 # set the size of each test set
n_splits = len(dataset) // window_size # calculate the number of splits we'll need

In [29]:
n_splits

1928

In [31]:
dataset.iloc[:, :45]

,city_resolvedAddress,day_datetime,isw_date_tomorrow_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,isw_0km,isw_0th,isw_10,isw_100
52886,"Чернігів, Україна",2022-02-25,2022-02-25,6.4,-1.7,2.1,-2.1,75.5,0.00,0.00,...,3.0,25.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0
121198,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,7.0,17.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0
121199,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,4.0,17.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0
121200,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,4.0,17.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0
121201,"Рівне, Україна",2022-02-25,2022-02-25,6.9,-0.6,2.4,-0.9,80.0,0.01,4.17,...,4.0,17.0,0.0,1.0,0.000000,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69594,"Одеса, Україна",2023-01-20,2023-01-20,9.7,4.2,8.1,5.2,82.0,0.00,0.00,...,4.0,15.0,2.0,7.0,6.881944,0.0,0.0,0.0,0.0,0.0
69593,"Одеса, Україна",2023-01-20,2023-01-20,9.7,4.2,8.1,5.2,82.0,0.00,0.00,...,3.0,15.0,2.0,0.0,5.881944,0.0,0.0,0.0,0.0,0.0
69592,"Одеса, Україна",2023-01-20,2023-01-20,9.7,4.2,8.1,5.2,82.0,0.00,0.00,...,3.0,15.0,2.0,0.0,4.881944,0.0,0.0,0.0,0.0,0.0
16631,"Кропивницький, Україна",2023-01-20,2023-01-20,11.9,3.2,7.0,3.3,78.2,0.00,0.00,...,3.0,11.0,2.0,24.0,11.530278,0.0,0.0,0.0,0.0,0.0
